## Water Hub Data Extraction
Manipulation of Santiago Botero's script for accessing the Columbia Basin Water Hub data.
This is a simplified version of "Water_Hub_Data_Extraction"
Dec 23, 2020  T.V.

#### Check that we can connect to the water hub repository
The cell below uses "requests" to connect with the water hub. A response of 200 tells us that the connection was successful.

In [1]:
# First we need to import the packages we need to make an HTTP request
import requests
import json

data = requests.get('https://waterhub.livinglakescanada.ca/api/3')
print(data)

<Response [200]>


#### Look at the packages available
The reply is a json file that is contains everything, i.e. all of the packages.
json is a javescript notation for moving and fetching data.
For more than you even want to know see:  https://docs.python.org/3/library/json.html


In [5]:
data = requests.get('https://waterhub.livinglakescanada.ca/api/3/action/package_search').json()
#print(data)                #uncomment to see the full json string

#### Make the output readable.
The data that json returns is a dictionary. That is hard to read. The cell below "prettifies" it.
The second cell sorts the file alphabetically/

In [6]:
#print (json.dumps(data, indent=1))    #uncomment to see the result

In [7]:
#print (json.dumps(data, indent=1, sort_keys=True))             #uncomment to see the pretty version


#### Search for a package
Here we specify which package within the above has the word Norns Creek data. (package_search?q=norns).
I have indented the print to 5 to make it easy to see the "results" list, within which we have two "resources" being
'named' 'Monitoring Stations GPS points' and 'Norns Creek Measurements'.
We want the id of the latter which is 12 lines above "name: Norns Creek Measuremenst" and is "id: fb4c4973-ad6e-4774-9832-c46c49f20369"


In [8]:
data = requests.get('https://waterhub.livinglakescanada.ca/api/3/action/package_search?q=norns').json()
#print(json.dumps(data, indent=5))   #uncomment to see the result

#### Finally, look at the actual data
If we look at the 'resource_id' of the "Norns Creek Measurements", which we put into the URL
--- > ~/datastore_search?resource_id=fb4c4973-ad6e-4774-9832-c46c49f20369' we get the JSON string.
I have printed this with only an indent of 2 because it is easy to read.
I "typed" it to prove that it is a string, not a list nor a dictionary.

In [9]:
data = requests.get('https://waterhub.livinglakescanada.ca/api/3/action/datastore_search?resource_id=fb4c4973-ad6e-4774-9832-c46c49f20369').json()

# Find the key for dictionary:
json_string = json.dumps(data['result']['records'], indent=2)
#print(type(json_string),json_string)    #uncomment to see the 4 station data dictionaries



#### Now we have the data
First we must convert it from a string to a python object, a list, using the Python method 'loads'
We can iterate over a list to pull out the dictionaries within it. Remember that Python indices start at 0.
We can install some QA/QC checks here such as each dictionary is the same length and key names. (secomd cell)

In [11]:
dict_list = json.loads(json_string)
print (type(dict_list))                   #confirm that we have a 'list'

for item in dict_list:                    #iterate and print because the list is only 4 items.
    #print (item)             #uncomment to see the 4 dictionaries
    continue

<class 'list'>


In [10]:
# ------------QA/QC---------------------
for item in dict_list:                  
    print (len(item))               # The dictionaries should be the same size                    
    print (item.keys())             # The fienld names should be the same.

20
dict_keys(['_id', 'Title', 'StnID', 'BankfulWidth', 'unit', 'FlowingDepthAvg', 'unit.1', 'StagnationDepthAvg', 'unit.2', 'DO', 'unit.3', 'pH', 'Temperature', 'unit.4', 'Turbidity', 'unit.5', 'WettedWidth', 'unit.6', 'slope', 'unit.7'])
20
dict_keys(['_id', 'Title', 'StnID', 'BankfulWidth', 'unit', 'FlowingDepthAvg', 'unit.1', 'StagnationDepthAvg', 'unit.2', 'DO', 'unit.3', 'pH', 'Temperature', 'unit.4', 'Turbidity', 'unit.5', 'WettedWidth', 'unit.6', 'slope', 'unit.7'])
20
dict_keys(['_id', 'Title', 'StnID', 'BankfulWidth', 'unit', 'FlowingDepthAvg', 'unit.1', 'StagnationDepthAvg', 'unit.2', 'DO', 'unit.3', 'pH', 'Temperature', 'unit.4', 'Turbidity', 'unit.5', 'WettedWidth', 'unit.6', 'slope', 'unit.7'])
20
dict_keys(['_id', 'Title', 'StnID', 'BankfulWidth', 'unit', 'FlowingDepthAvg', 'unit.1', 'StagnationDepthAvg', 'unit.2', 'DO', 'unit.3', 'pH', 'Temperature', 'unit.4', 'Turbidity', 'unit.5', 'WettedWidth', 'unit.6', 'slope', 'unit.7'])


### Update to get an array which we can write to a file or further process

In the cell above we printed the dictionary keys and confirmed that each dictionary contained the same number of items (20) and the same keys. 

Next we take the names of first dictionary, dict_list[0], as an iterable list so that we are always lookiing through the dictionaries in the same order. This is necessary because dictionaries store their data in random order.

In [13]:
#convert the dictionary keys to a list
names = dict_list[0]
column_names = list(names)
print (type(column_names), '\n', column_names)

<class 'list'> 
 ['_id', 'Title', 'StnID', 'BankfulWidth', 'unit', 'FlowingDepthAvg', 'unit.1', 'StagnationDepthAvg', 'unit.2', 'DO', 'unit.3', 'pH', 'Temperature', 'unit.4', 'Turbidity', 'unit.5', 'WettedWidth', 'unit.6', 'slope', 'unit.7']


### Loop through the fields and pick out the values
The "dict_list" contains 4 dictionaries but we will set it up so that it will handle any number of dictionaries sing a counter "i". Then we will set up an empty array "array" to hold rows of values arranged according to the same dictionary key in each row.

The first "for" loop looks at each set of data items (dictionary) in turn. The nested "for" loop then looks at each item in teh dictionary and extracts the value using our  list of "column_names" to keep the values in order. Finally we print the array of values to confirm that we have created a 2D array.


In [14]:
i = 0
array = []
for item in dict_list:
    row = []
    for name in column_names:
        val = item.get(name, 'key not found')  # default message if the key does not exist
        row.append(val)
    #print (row)
    array.append(row)
    i += 1
#print (array)           #uncomment to see the 2D array
        

### Convert the array to a pandas dataframe
A pandas dataframe can be easily written to a csv file as shown in the commented out final line.

or can be further manipulated in Python to process the data.

In [29]:
import pandas as pd
dataframe=pd.DataFrame(array, columns=(column_names))
print (dataframe)

#dataframe.to_csv('file_name.csv')

   _id     Title  StnID  BankfulWidth unit  FlowingDepthAvg unit.1  \
0    0  Station1      1          0.10    m             0.60      m   
1    1      Stn2      2          0.93    m             0.53      m   
2    2      Stn3      3          0.87    m             0.43      m   
3    3      Stn4      4          0.80    m             0.54      m   

   StagnationDepthAvg unit.2    DO unit.3    pH  Temperature unit.4  \
0                0.80      m  7.00   mg/L  6.00         6.00      C   
1                0.58      m  6.34   mg/L  6.45         8.30      C   
2                0.54      m  6.54   mg/L  6.38         6.76      C   
3                0.60      m  6.45   mg/L  7.00         7.40      C   

   Turbidity unit.5  WettedWidth unit.6  slope unit.7  
0        1.0    NTU         0.90      m      4      %  
1        1.3    NTU         0.89      m      5      %  
2        1.2    NTU         0.82      m      4      %  
3        1.1    NTU         0.73      m      3      %  
